# Getting the Budget and Revenue from the-numbers.com

###  Using beautiful soup to scrape the website to get the data required

Here we are scraping the Budget and the Revenue first with the index number

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

url = "http://www.the-numbers.com/movie/budgets/all"
r=requests.get(url)
data=r.text
soup=bs(data,"html.parser")
each=soup.find_all('td',class_="data")

mydata=[]
for i in each:
    i=i.getText()
    mydata.append(i)
print(mydata)


In [ ]:
length=len(mydata)/4
length=int(length)
print(length)

In [ ]:
import numpy as np
PRICES=np.reshape(mydata,(length,4))

### Putting the scarped data into a pandas dataframe

In [ ]:
import pandas as pd
heading1=['index','budget','domestic','worldwide']
pricelist = pd.DataFrame(PRICES, columns=heading1)
print(pricelist)

### Now we are scraping the name and dates of the movies and putting them into a pandas dataframe

In [ ]:
names=soup.findAll('a')
myname=[]

for name in names:
    name=name.getText()
    myname.append(name)

names2=[]    
for i in myname[50:(50+2*length)]:
    names2.append(i)
    
NAMES=np.reshape(names2,(length,2))
print(NAMES)

In [ ]:
import pandas as pd
heading2=['date','name']
namelist = pd.DataFrame(NAMES, columns=heading2)
print(namelist)

### We join them along the index axis and get the final database withe the date of release, the name of the movie, Budget, WorldWide Rev and Domestic Rev

In [ ]:
myfilm = pd.concat([namelist, pricelist], axis=1, join='outer')

In [ ]:
myfilm.head(10)

### Selecting the top 100 movies in the year 2015 in terms of Revenue and writing them into a csv file

In [ ]:
myfilm = myfilm[myfilm['date'].str.contains("2015")]

In [ ]:
del myfilm["index"]

In [ ]:
del myfilm["domestic"]

In [ ]:
del myfilm["date"]

In [ ]:
myfilm['worldwide'] = myfilm['worldwide'].map(lambda x: x.lstrip('$'))

In [ ]:
myfilm.head(5)

In [ ]:
myfilm['worldwide'] = myfilm['worldwide'].str.replace(',','')

In [ ]:
myfilm['budget'] = myfilm['budget'].map(lambda x: x.lstrip('$'))

In [ ]:
myfilm['budget'] = myfilm['budget'].str.replace(',','')

In [ ]:
myfilm.head(10)

In [ ]:
myfilm.to_csv('worldwide.csv')

In [ ]:
!csvcut worldwide.csv |head |csvlook

In [ ]:
!sort -k4 -rn -t, worldwide.csv | csvcut -c2,3,4  > Budget_Revenue.csv